In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import pandas as pd
import os
from pathlib import Path

import itertools as it
from collections import Counter
import numpy as np

from aiida import load_profile
from aiida.orm import load_node

from pymatgen.io.ase import AseAtomsAdaptor

from pybat import Cathode

from ase.build.tools import sort

from ase.io.espresso import read_espresso_in

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

import itables.options as opt

opt.style = "table-layout:auto;width:80%:left"
opt.columnDefs = [{"className": "dt-left", "targets": "_all"}]

pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None

In [ ]:
load_profile()

# Olivine directory tree
<!-- 
├── Bulk_Li
├── LixFePO4
│   ├── Li0.00
│   ├── Li0.25
│   ├── Li0.50
│   ├── Li0.75
│   └── Li1.00
├── LixMn0.5Fe0.5PO4
│   ├── Li0.00
│   ├── Li0.25
│   ├── Li0.50
│   ├── Li0.75
│   └── Li1.00
├── LixMnPO4
│   ├── Li0.00
│   ├── Li0.25
│   ├── Li0.50
│   ├── Li0.75
│   └── Li1.00
└── Pseudopotentials
    ├── PseudoDojo
    └── SSSP
-->

# Spinel directory tree

<!--
├── LixMn1.5Ni0.5O4
│   ├── DFT+U
│   │   ├── LiMn1.5Ni0.5O4
│   │   └── Mn1.5Ni0.5O4
│   └── DFT+U+V
│       ├── LiMn1.5Ni0.5O4
│       └── Mn1.5Ni0.5O4
└── LixMn2O4
    ├── DFT+U
    │   ├── LiMn2O4
    │   └── Mn2O4
    └── DFT+U+V
        ├── LiMn2O4
        └── Mn2O4
-->

# Read in fully lithiated phospho-olivine and spinel structures from pw inputs

In [ ]:
project_dir = '/home/jgeiger/projects/bat_uv_ml/'

# Read in csv with pandas
iurii_full_df = pd.read_csv(os.path.join(project_dir, 'data', 'iurii_full_data_df.csv'))

In [ ]:
iurii_full_df

***
# Fully lithiated structures

In [ ]:
fully_lithiated_df = iurii_full_df.loc[
    (iurii_full_df['abs_path'].str.contains('1_vcrelax'))
 &  (iurii_full_df['abs_path'].str.contains('Li1.00') | iurii_full_df['abs_path'].str.contains('LiMn'))
 &  (iurii_full_df['calc_in'].str.contains('vcrelax.in') | iurii_full_df['calc_in'].str.contains('vcrelax.1.in'))
 & ~(iurii_full_df['abs_path'].str.contains('DFT+U+V', regex=False) | iurii_full_df['abs_path'].str.contains('DFT_plus_', regex=False))
]

fully_lithiated_df.shape
fully_lithiated_df = fully_lithiated_df.reset_index(drop=False)

# ! Replace Mn3u and other special labels with atom types for reading in QE input structure with ASE
fully_lithiated_df['clean_input'] = fully_lithiated_df['abs_path_in'].apply(lambda x: Path(x).read_text())
fully_lithiated_df['clean_input'] = fully_lithiated_df['clean_input'].str.replace('\nM\d[u,d] ', '\nMn ', regex=True)
fully_lithiated_df['clean_input'] = fully_lithiated_df['clean_input'].str.replace('\nO\d ', '\nO ', regex=True)
fully_lithiated_df['clean_input_file'] = fully_lithiated_df['abs_path_in'].apply(lambda x: x.replace('.in', '.clean.in'))
_ = fully_lithiated_df.apply(lambda x: Path(x['clean_input_file']).write_text(x['clean_input']), axis=1)

# ! Actually read in the input files with ASE
fully_lithiated_df['ase_in'] = fully_lithiated_df['clean_input_file'].apply(lambda x: read_espresso_in(x))
fully_lithiated_df['ase_in'] = fully_lithiated_df['ase_in'].apply(lambda x: sort(x, tags=x.numbers))
# devel_df['ase_out'] = devel_df.apply(lambda x: read_espresso_out(os.path.join(x['abs_path'], x['calc_out'])), axis=1)

# ! Get chemical formula and symbols
fully_lithiated_df['chem_formula'] = fully_lithiated_df['ase_in'].apply(lambda x: x.get_chemical_formula())
fully_lithiated_df['chem_symbols'] = fully_lithiated_df['ase_in'].apply(lambda x: x.get_chemical_symbols())

# type(devel_df['ase_out'].values[0])
# ! Convert ase structures to pymatgen
fully_lithiated_df['pmg_in'] = fully_lithiated_df['ase_in'].apply(lambda x: AseAtomsAdaptor.get_structure(x))
# devel_df['pmg_out'] = devel_df['ase_out'].apply(lambda x: AseAtomsAdaptor.get_structure(x))

In [ ]:
show_columns_structures = ['abs_path', 'calc_in', 'calc_out', 'calc_type', 'chem_formula', 'short_name']
short_names = ['spinel_LiMnNiO', 'spinel_LiMnO', 'olivine_LiFePO', 'olivine_LiMnFePO', 'olivine_LiMnPO']
fully_lithiated_df['short_name'] = short_names
fully_lithiated_df[show_columns_structures].head()
# fully_lithiated_df['calc_in'].values

***
# Sampling of configurations using `icet`

In [ ]:
# 'spinel_LiMnNiO': default_dict,   # Li8Mn12Ni4O32
# 'spinel_LiMnO': default_dict,     # Li8Mn16O32
# 'olivine_LiFePO': default_dict,   # Li4Fe4P4O16
# 'olivine_LiMnFePO': default_dict, # Li4Mn2Fe2P4O16
# 'olivine_LiMnPO4': default_dict,  # Li4Mn4P4O16

olivine_names = ['olivine_LiFePO', 'olivine_LiMnFePO', 'olivine_LiMnPO']
olivine_df_keys = list(it.product(olivine_names, [1,2]))
spinel_names = ['spinel_LiMnNiO', 'spinel_LiMnO']
spinel_df_keys = list(it.product(spinel_names, [1,]))
sampling_df_keys = olivine_df_keys + spinel_df_keys

In [ ]:
def create_sampling_df(input_pmg, **kwargs):
    # ! Lithiation currently works only for one size

    initial_cat = Cathode.from_structure(input_pmg)
    configuration_list = initial_cat.get_cation_configurations(**kwargs)

    pmg_list = [cathode.as_ordered_structure() for cathode in configuration_list]
    ase_list = [AseAtomsAdaptor.get_atoms(pmg_) for pmg_ in pmg_list]
    formula_list = [_.get_chemical_formula() for _ in ase_list]
    cell_list = [np.array2string(_.cell.todict()['array']) for _ in ase_list]
    sg_list = [_.space_group for _ in configuration_list]
    li_list = [_.get_chemical_symbols().count('Li') for _ in ase_list]

    # ? Still need to apply the multiplicity_dict
    li_multiplicity_dict = dict(Counter(li_list))
    cell_multiplicity_dict = dict(Counter(cell_list))
    sg_multiplicity_dict = dict(Counter(sg_list))


    # Put all the data into pandas df
    return_df = pd.DataFrame()

    return_df['configuration'] = configuration_list
    return_df['li_number'] = li_list
    return_df['formula'] = formula_list
    return_df['cell'] = cell_list
    return_df['space_group'] = sg_list
    return_df['ase_structure'] = ase_list

    return_df['lithiation'] = return_df['li_number'] / (len(kwargs['substitution_sites'])*kwargs['sizes'][0])

    # return_df = return_df.sort_values(by='lithiation', ascending=False)
    return_df['li_mpc']= return_df['li_number'].map(li_multiplicity_dict)
    return_df['cell_mpc']= return_df['cell'].map(cell_multiplicity_dict)
    return_df['sg_mpc']= return_df['space_group'].map(sg_multiplicity_dict)

    return return_df

conf_df_list = []

for structure_index, sampling_df_key in enumerate(sampling_df_keys):
    loop_pmg = fully_lithiated_df.loc[fully_lithiated_df['short_name'] == sampling_df_key[0]]['pmg_in'].values[0]
    loop_ase = AseAtomsAdaptor.get_atoms(loop_pmg)
    loop_ase.center()

    size = sampling_df_key[1]
    
    kwargs_dict = dict(
        substitution_sites=range(0, loop_ase.get_chemical_symbols().count('Li')),
        cation_list=["Li", "Vac"],
        sizes=[size],
        concentration_restrictions={"Li": (0, 1)},
        max_configurations=None,
    )
    conf_df = create_sampling_df(input_pmg=loop_pmg, **kwargs_dict)
    conf_df_list.append(conf_df)

sampling_df_dict = dict(zip(sampling_df_keys, conf_df_list))

In [ ]:
show_columns_sampling = ['li_number', 'formula', 'space_group', 'lithiation', 'li_mpc', 'cell_mpc', 'sg_mpc']

pd.set_option('display.max_rows', 10)
for sampling_df_key in sampling_df_keys[:2]:
    current_df = sampling_df_dict[sampling_df_key][show_columns_sampling]
    print("Structure: {}, Size: {}, # Configs: {}".format(*sampling_df_key, current_df.shape[0]))
    current_df
    # view(sampling_df_dict[sampling_df_key]['ase_structure'].values)
    # break

***
### Bash cell for verdi stuff

In [ ]:
%%bash
verdi process list -a -p 12

In [ ]:
%%bash
%%capture
verdi calcjob inputls 2176    # shows the list of input files
verdi calcjob inputcat 2176   # shows the input file of the calculation
verdi calcjob outputls 2176   # shows the list of output files
verdi calcjob res 2176        # shows the parser results of the calculation

In [ ]:
%%bash
verdi process show 2176
verdi process report 2176

In [ ]:
%%bash
verdi devel --help